In [ ]:
'''

以先前案例組合而成

S3 客戶端生成
DynamoDB 客戶端生成
flask app 生成

創建S3 桶子
創建 DynamoDB Table

第一個方法
    用戶訪問網站
    看見上傳網站頁面，下方有目前已上傳圖片的表格清單，表格內要有 S3 的連結名

第二個方法
    用戶上傳圖片
    伺服器收到 Request
    生成 presigned url，上傳的檔案預設需公開
    並插入資料至 DynamoDB
    讓用戶將圖片傳至 presigned url
    看見上傳網站頁面，下方有目前已上傳圖片的表格清單，表格內要有 S3 的連結名


啟動flask app，使外部port 5000 能訪問


'''

In [ ]:
!pip install boto3 awscli requests flask

In [1]:
'''

S3 客戶端生成
DynamoDB 客戶端生成
flask app 生成

'''
import boto3
from flask import Flask


s3_client = boto3.client('s3', endpoint_url='http://localstack-main:4566')
# s3_client = boto3.client('s3')
dynamodb = boto3.resource('dynamodb', endpoint_url='http://localstack-main:4566')

app = Flask(__name__)


In [2]:
'''

創建S3 桶子
創建 DynamoDB Table

'''


create_bucket_response = s3_client.create_bucket(
    Bucket='cxcxc-aws-certificate',
)
print(create_bucket_response)


table = dynamodb.create_table(
    TableName='s3-objects-list', 
    KeySchema=[
        {
            'AttributeName': 'image_name',
            'KeyType': 'HASH'  # Partition key
        },
        {
            'AttributeName': 'object_name',
            'KeyType': 'RANGE'  # Sort key
        }
    ], 
    AttributeDefinitions=[
        {
            'AttributeName': 'image_name',
            'AttributeType': 'S'  # N 代表數字； S 代表字串。
        },
        {
            'AttributeName': 'object_name',
            'AttributeType': 'S'  
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

print(table)


{'ResponseMetadata': {'RequestId': '5D754175430594D2', 'HostId': 'MzRISOwyjmnup5D754175430594D27/JypPGXLh0OVFGcJaaO3KW/hRAqKOpIEEp', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/xml; charset=utf-8', 'content-length': '175', 'access-control-allow-origin': '*', 'last-modified': 'Tue, 18 Aug 2020 16:08:01 GMT', 'x-amz-request-id': '5D754175430594D2', 'x-amz-id-2': 'MzRISOwyjmnup5D754175430594D27/JypPGXLh0OVFGcJaaO3KW/hRAqKOpIEEp', 'access-control-allow-methods': 'HEAD,GET,PUT,POST,DELETE,OPTIONS,PATCH', 'access-control-allow-headers': 'authorization,content-type,content-md5,cache-control,x-amz-content-sha256,x-amz-date,x-amz-security-token,x-amz-user-agent,x-amz-target,x-amz-acl,x-amz-version-id,x-localstack-target,x-amz-tagging', 'access-control-expose-headers': 'x-amz-version-id', 'connection': 'close', 'date': 'Tue, 18 Aug 2020 16:08:01 GMT', 'server': 'hypercorn-h11'}, 'RetryAttempts': 0}}
dynamodb.Table(name='s3-objects-list')


In [ ]:
# 列出現有 bucket
response = s3_client.list_buckets()
print(response.get('Buckets'))

# 列出現有 table
table_list = list(dynamodb.tables.all())
print(table_list)


In [ ]:
'''

掃描DynamoDB Table 所有項目

'''

def scan_dynamodb(table_name='s3-objects-list') -> list:
    
    dynamodb = boto3.resource('dynamodb', endpoint_url='http://localstack-main:4566')
    table = dynamodb.Table(table_name)

    # 掃描 DynamoDB 將所有 image 列出
    response = table.scan(
        FilterExpression=Attr('image_name').exists()
    )
    items = response['Items']
    
    return items


In [3]:
"""

根據時間為物件取名
生成S3 presigned post url

透過前端頁面上傳至S3 presigned post url

"""
import datetime

def create_presigned_post(bucket_name='cxcxc-aws-certificate', object_name=None) -> dict:
    
    s3_client = boto3.client('s3', endpoint_url='http://localstack-main:4566')
#     s3_client = boto3.client('s3')
    if object_name is None:
        object_name = datetime.datetime.utcnow().strftime('%Y%m%d_%H%M%S')
    
    # Generate a presigned S3 POST URL
    response = s3_client.generate_presigned_post(bucket_name, object_name)
    
    # The response contains the presigned URL and required fields
    return response


@app.route('/upload-images', methods=['GET', 'POST'])
def upload_images():
    
    presigned_post = create_presigned_post()
    print(presigned_post)
    presigned_post_url = presigned_post.get('url')
    required_fields = presigned_post.get('fields')

    html = f"""<html>
      <head>
        <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
      </head>
      <body>
        <!-- 因為我們在container 外的瀏覽器連入，所以S3 的URL 要使用虛擬機IP，以下這行是原本的用法
        <form action="{presigned_post_url}" method="post" enctype="multipart/form-data"> 
        -->
        
        <form action="http://192.168.71.129:4566/cxcxc-aws-certificate" method="post" enctype="multipart/form-data">
        
          <input type="hidden" name="key" value="{required_fields.get('key')}" />
          <input type="hidden" name="AWSAccessKeyId" value="{required_fields.get('AWSAccessKeyId')}" />
          <input type="hidden" name="policy" value="{required_fields.get('policy')}" />
          <input type="hidden" name="signature" value="{required_fields.get('signature')}" />
        File:
          <input type="file"   name="file" /> <br />
          <input type="submit" name="submit" value="Upload to Amazon S3"  />
        </form>
        
        
      </body>
    </html>
    """
    
    return html



In [ ]:
# @app.route('/images', methods=['GET'])
def list_images():
    
    dynamodb = boto3.resource('dynamodb', endpoint_url='http://localstack-main:4566')
    
    table = dynamodb.Table('s3-objects-list')

    # 掃描 DynamoDB 將所有 image 列出
    response = table.scan(
        FilterExpression=Attr('image_name').exists()
    )
    items = response['Items']
    
    print(items)
    
    html = f"""<!doctype html>
        <title>Read Files</title>
        <h1>Read Files</h1>

        <table border="1">
            <tr>
                <td>已上傳的圖片名稱</td>
                <td>S3 物件名稱</td>
                <td>S3 URL</td>
            </tr>

                <tr>
                    <td>{{ uploaded_images[i] }}</td>
                    <td>{{ images_url[i] }}</td>
                </tr>

        </table>
    """
    
    return items

In [ ]:
list_images()

In [ ]:
'''

第一個方法
    用戶訪問網站
    看見上傳網站頁面，下方有目前已上傳圖片的表格清單，表格內要有 S3 的連結名

'''

In [ ]:
'''

第二個方法
    用戶上傳圖片
    伺服器收到 Request
    生成 presigned url，上傳的檔案預設需公開
    並插入資料至 DynamoDB
    讓用戶將圖片傳至 presigned url
    看見上傳網站頁面，下方有目前已上傳圖片的表格清單，表格內要有 S3 的連結名

'''



In [ ]:
'''

啟動flask app，使外部port 5000 能訪問

'''

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.19.0.6 - - [18/Aug/2020 16:17:47] "GET /upload-images HTTP/1.1" 200 -


{'url': 'http://localstack-main:4566/cxcxc-aws-certificate', 'fields': {'key': '20200818_161747', 'AWSAccessKeyId': 'abc', 'policy': 'eyJleHBpcmF0aW9uIjogIjIwMjAtMDgtMThUMTc6MTc6NDdaIiwgImNvbmRpdGlvbnMiOiBbeyJidWNrZXQiOiAiY3hjeGMtYXdzLWNlcnRpZmljYXRlIn0sIHsia2V5IjogIjIwMjAwODE4XzE2MTc0NyJ9XX0=', 'signature': 'ir1QQRFyWuOvYzdKXuemWSTB5GQ='}}


172.19.0.6 - - [18/Aug/2020 16:19:29] "GET /upload-images HTTP/1.1" 200 -


{'url': 'http://localstack-main:4566/cxcxc-aws-certificate', 'fields': {'key': '20200818_161929', 'AWSAccessKeyId': 'abc', 'policy': 'eyJleHBpcmF0aW9uIjogIjIwMjAtMDgtMThUMTc6MTk6MjlaIiwgImNvbmRpdGlvbnMiOiBbeyJidWNrZXQiOiAiY3hjeGMtYXdzLWNlcnRpZmljYXRlIn0sIHsia2V5IjogIjIwMjAwODE4XzE2MTkyOSJ9XX0=', 'signature': 'PX0QgCQwVDiklv7jUS5blHCqCEk='}}


In [5]:
s3_client.list_objects_v2(Bucket='cxcxc-aws-certificate')

{'ResponseMetadata': {'RequestId': '3941C709B13DAA69',
  'HostId': 'MzRISOwyjmnup3941C709B13DAA697/JypPGXLh0OVFGcJaaO3KW/hRAqKOpIEEp',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/xml; charset=utf-8',
   'content-length': '461',
   'access-control-allow-origin': '*',
   'last-modified': 'Tue, 18 Aug 2020 16:10:17 GMT',
   'x-amz-request-id': '3941C709B13DAA69',
   'x-amz-id-2': 'MzRISOwyjmnup3941C709B13DAA697/JypPGXLh0OVFGcJaaO3KW/hRAqKOpIEEp',
   'accept-ranges': 'bytes',
   'content-language': 'en-US',
   'cache-control': 'no-cache',
   'content-encoding': 'identity',
   'access-control-allow-methods': 'HEAD,GET,PUT,POST,DELETE,OPTIONS,PATCH',
   'access-control-allow-headers': 'authorization,content-type,content-md5,cache-control,x-amz-content-sha256,x-amz-date,x-amz-security-token,x-amz-user-agent,x-amz-target,x-amz-acl,x-amz-version-id,x-localstack-target,x-amz-tagging',
   'access-control-expose-headers': 'x-amz-version-id',
   'connection': 'close',
  